In [1]:
from manim import *
from objects import *

In [39]:
%%manim -ql Test
from processing import *
from animations import *

dots_color=[RED, ManimColor("#2f9e44"), ManimColor("#228be6"), ManimColor("#a5d8ff"), GREY]
lines_color = [YELLOW, ORANGE, PURPLE]

class Test(Scene):
    def construct(self):
        BTree = BinaryTree(root=buildTree(dots_color, 7), x_start=0, x_distance=3, y_start=3 ,y_distance=1, dots_color=dots_color, lines_color=lines_color)
        BTree.build_structure(BTree.root)
        BTree.display(self)
        level_ordered_tags = BTree.get_level_ordered_tags()
        initial_dot = Dot()
        initial_dot.move_to(BTree.entry_dot.get_center())
        initial_dot.original_pos = BTree.entry_dot.get_center()
        self.play(FadeIn(initial_dot))
        self.play(initial_dot.animate.move_to(level_ordered_tags[0][0]))
        level_initial_dots = [VGroup(initial_dot)]
        height = BTree.root.get_height()

        for i in range(height):
            level_dots = VGroup()
            for a, dot in enumerate(level_initial_dots[i]):
                self.play(Indicate(dot), run_time=2)
                left_dot = Dot()
                left_dot.move_to(dot.get_center())
                left_dot.original_pos = left_dot.get_center()
                level_dots.add(left_dot)
                left_child_tag = level_ordered_tags[2**(i+1)+2*a-1][0]
                my_left_L = level_ordered_tags[2**(i)+a-1][1]
                left_dot.L = VMobject()
                left_dot.L.set_points_as_corners([my_left_L[0].start, my_left_L[0].end, my_left_L[1].end])
                left_dot.tracer = TracedPath(left_dot.get_center, stroke_color=YELLOW, stroke_width=4)
                self.add(left_dot.tracer)
                right_dot = Dot()
                right_dot.move_to(dot.get_center())
                right_dot.original_pos = right_dot.get_center()
                level_dots.add(right_dot)
                right_child_tag = level_ordered_tags[2**(i+1)+2*a][0]
                my_right_L = level_ordered_tags[2**(i)+a-1][2]
                right_dot.L = VMobject()
                right_dot.L.set_points_as_corners([my_right_L[0].start, my_right_L[0].end, my_right_L[1].end])
                right_dot.tracer = TracedPath(right_dot.get_center, stroke_color=YELLOW, stroke_width=4)
                self.add(right_dot.tracer)
            self.play(AnimationGroup(*[MoveAlongPath(dot, dot.L) for dot in level_dots]), run_time=2*(i+1))
            level_initial_dots.append(level_dots)
        

Manim Community v0.19.0

[05/25/25 16:30:43] INFO     Animation 0 : Using cached data (hash :                           ]8;id=351506;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=117376;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             1185818338_3779699502_223132457)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=171780;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=636381;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             624642324_2911954173_3976939568)                                                      

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=97385;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=135891;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             624642324_708762236_3127875305)                                                       

[05/25/25 16:30:44] INFO     Animation 3 : Using cached data (hash :                           ]8;id=266769;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=173474;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             624642324_1972434639_728265532)                                                       

...

[05/25/25 16:30:45] INFO     Animation 7 : Using cached data (hash :                           ]8;id=27555;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=574061;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             624642324_1977435482_4020624539)                                                      

                    INFO     Animation 8 : Using cached data (hash :                           ]8;id=433872;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=431869;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             624642324_305086370_131986917)                                                        

                    INFO     Combining to Movie file.                                      ]8;id=925405;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=143913;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=674893;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=368981;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#886\886]8;;\
                             File ready at 'C:\Users\Asser\Documents\Coding\VS                                     
                             CODE\Manim\BinaryTree\media\videos\BinaryTree\480p15\Test.mp4                         
                             '                                                                                     
                                                                                                                   

                    INFO     Rendered Test                                                             ]8;id=295322;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=892162;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene.py#255\255]8;;\
                             Played 9 animations                                                                   

In [11]:
%%manim -ql Test

class Test(Scene):
    def construct(self):
        # Define the L-shaped path using points
        start = LEFT * 4 + DOWN * 2
        corner = RIGHT * 2 + DOWN * 2
        end = RIGHT * 2 + UP * 2
     
        dot = Dot(color=YELLOW).move_to(start)
        # Create a VMobject path shaped like an L
        dot.path = VMobject()
        dot.path.set_points_as_corners([start, corner, end])

       

        # Trace the full movement of the dot
        dot.tracer = TracedPath(dot.get_center, stroke_color=BLUE, stroke_width=4)

        # Add to scene
        self.add(dot.tracer, dot)

        # Animate the dot along the L-shaped path
        self.play(MoveAlongPath(dot, dot.path), run_time=4, rate_func=linear)

        self.wait()

Manim Community v0.19.0

[05/25/25 15:46:34] INFO     Animation 0 : Partial movie file written in                   ]8;id=244504;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=634885;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\Asser\Documents\Coding\VS                                                   
                             CODE\Manim\BinaryTree\media\videos\BinaryTree\480p15\partial_                         
                             movie_files\Test\1185818338_2636999143_2036208256.mp4'                                

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=913108;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=208269;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\Asser\Documents\Coding\VS                                                   
                             CODE\Manim\BinaryTree\media\videos\BinaryTree\480p15\partial_                         
                             movie_files\Test\624642324_2449613945_1757829693.mp4'                                 

                    INFO     Combining to Movie file.                                      ]8;id=413574;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=582116;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=278007;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=966901;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#886\886]8;;\
                             File ready at 'C:\Users\Asser\Documents\Coding\VS                                     
                             CODE\Manim\BinaryTree\media\videos\BinaryTree\480p15\Test.mp4                         
                             '                                                                                     
                                                                                                                   

                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=707917;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=280103;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#856\856]8;;\
                             manim has removed the 2 oldest file(s). You can change this                           
                             behaviour by changing max_files_cached in config.                                     

                    INFO     Rendered Test                                                             ]8;id=723141;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=688743;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene.py#255\255]8;;\
                             Played 2 animations                                                                   